# Solutions to Problem Set 0 — Coding Part

**Course:** Data Compression With And Without Deep probabilistic models (Prof. Bamler at University of Tübingen)

- This notebook shows suggested solutions to the coding part of Problem Set 0, published and discussed on 19 April 2023.
- You can download the original problem set from the [course website](https://robamler.github.io/teaching/compress23/).

## Problem 0.3: Naive Symbol Code Implementation

In this exercise, we'll implement a very naive but correct encoder and decoder for prefix-free symbol codes.
We only care about correctness for now, not about computational efficiency.

We represent bit strings (i.e., code words and the concatenated encoded message) as lists of boolean values, where `True` represents a "one"-bit and `False` represents a "zero" bit.
Please be aware that this would be an extremely inefficient representation for a real application.
We represent code books as dictionaries from symbols to bit strings.

### Sample Code Books

Our decoding algorithm will only work with prefix codes.
Let's define some sample prefix codes for our unit tests.

In [1]:
# Our example C^{(3)} from Problem 0.2
EXAMPLE_CODEBOOK_SGOM_C3 = {
    2: [False, True, False],
    3: [True, False],
    4: [False, False],
    5: [True, True],
    6: [False, True, True]
}

# additional example (exercise: verify that this is a prefix code)
EXAMPLE_CODEBOOK2 = {
    'a': [True, False],
    'b': [False],
    'c': [True, True, False, False],
    'd': [True, True, False, True],
    'e': [True, True, True],
}

### (a) Encoder

The encoder is very simple.
Fill in the blank where it says "TODO" (a single line of code will do).

In [2]:
def encode(message, codebook):
    """Encodes a sequence of symbols using a uniquely decodable symbol code.

    This is a very inefficient implementation for teaching purpose only.

    Args:
        message (list): The message you want to encode, as a list of symbols.
        codebook (dict): A codebook for a uniquely decodable symbol code. Must
            be a dictionary whose keys contain all symbols that appear in
            `message` (it may contain additional keys). Each key must map to a
            list of booleans, representing the code word as a sequence of bits.

    Returns:
        list: The encoded bit string as a list of bools.
    """
    
    encoded = []
    
    for symbol in message:
        # TODO: look up the code word for `symbol` in the `codebook` and append
        # it to `encoded` (1-2 lines of code)
        #
        # PROPOSED SOLUTION:
        encoded += codebook[symbol]
    
    return encoded

Now run these unit tests to verify your implementation:

In [3]:
assert encode([], EXAMPLE_CODEBOOK_SGOM_C3) == []
assert encode([], EXAMPLE_CODEBOOK2) == []
assert (
    encode([4, 3, 6, 4, 2], EXAMPLE_CODEBOOK_SGOM_C3)
    == [False, False, True, False, False, True, True, False, False, False, True, False]
)
assert (
    encode(['c', 'b', 'a', 'd', 'b', 'b', 'd', 'e'], EXAMPLE_CODEBOOK2)
    == [True, True, False, False, False, True, False, True, True, False, 
        True, False, False, True, True, False, True, True, True, True]
)

### (b) Decoder

The decoder is more complicated because it has to infer the boundaries between concatenated code words.
To do this, we will use the assumption that the code book defines a *prefix-free* symbol code (aka a *prefix code*).

We use a kind of brute-force implementation here.
It is correct but very inefficient.
We'll implement a more efficient method for a specific type of prefix codes in Problem 2.5 of Problem Set 2.

Fill in the blanks where it says "TODO".

In [4]:
def decode(encoded, codebook):
    """Decodes a bitstring into a sequence of symbols using a prefix-free symbol code.

    This is a very inefficient implementation for teaching purposes only.

    Args:
        encoded (list): The compressed bit string as a list of bools.
        codebook (dict): A codebook for a prefix-free symbol code.

    Returns:
        list: The decoded message as a list of symbols.
    """
    
    # Strategy:
    # - Initialize an empty list `decoded`, in which we'll accumulate the
    #   decoded symbols.
    # - Iterate through the bit string `encoded`, starting at `position = 0`;
    #   at each `position`, iterate through the code words in the code book and
    #   check if `encoded[position:]` starts with this code word.
    # - If we find a match, append the symbol to `decoded` and advance
    #   `position` by the length of the code word. If not, raise an exception.

    decoded = []
    position = 0
    
    while position != len(encoded):
        found_symbol = False

        for symbol, codeword in codebook.items():
            # TODO: implement the loop body (about 5 lines of code);
            # in detail, check if we found the right symbol; if so, append it
            # to `decoded`, advance `position`, set `found_symbol` to `True`,
            # and `break` out of the `for` loop.
            #
            # PROPOSED SOLUTION:
            if encoded[position:position+len(codeword)] == codeword:
                found_symbol = True
                decoded.append(symbol)
                position += len(codeword)
                break

        if not found_symbol:
            raise ValueError("no symbol matches")
        
    return decoded

Now run these unit tests to verify your implementation:

In [5]:
assert decode([], EXAMPLE_CODEBOOK_SGOM_C3) == []
assert decode([], EXAMPLE_CODEBOOK2) == []
assert decode(
    [False, False, True, False, False, True, True, False, False, False, True, False],
    EXAMPLE_CODEBOOK_SGOM_C3) == [4, 3, 6, 4, 2]
assert decode(
    [True, True, False, False, False, True, False, True, True, False, 
     True, False, False, True, True, False, True, True, True, True],
    EXAMPLE_CODEBOOK2) == ['c', 'b', 'a', 'd', 'b', 'b', 'd', 'e']

### (c) Why Only Prefix Codes?

Why do we require the code to be prefix free?
Wouldn't unique decodability suffice?

The cell below defines code book $C^{(5)}$ from Problem 0.2, which is uniquely decodable but *not* prefix free.
Come up with a message `example_message` such that encoding it and then decoding it with $C^{(5)}$ fails (i.e., such that the line saying `decoded = decode(...)` below raises an exception).

In [6]:
# Our example C^{(5)} from Problem 0.2
EXAMPLE_CODEBOOK_SGOM_C5 = {
    2: [False, True, False],
    3: [False, True],
    4: [False, False],
    5: [True, True],
    6: [True, True, False]
}

# TODO: find some `example_message`s such that the code below fails.
#
# PROPOSED SOLUTION:
# It is difficult to come up with an example that reproducibly fails every
# time. How `decode` operates on a non-prefix code depends on the order in which
# the inner loop iterates over the code book. Since the code book is a hash
# map, this order is unspecified and may depend on various things such as the
# python version. However, among the following two examples, at least one
# should always fail since one will fail if the inner loop iterates over
# `symbol = 2` before `symbol = 3` and the other will fail if the opposite is
# the case.

example_message1 = [3, 2]
example_message2 = [2, 4]

# These two should still work:
encoded1 = encode(example_message1, EXAMPLE_CODEBOOK_SGOM_C5)
encoded2 = encode(example_message2, EXAMPLE_CODEBOOK_SGOM_C5)

# At least one of these should raise an exeption:
decoded1 = decode(encoded1, EXAMPLE_CODEBOOK_SGOM_C5) # <-- Either this ...
decoded2 = decode(encoded2, EXAMPLE_CODEBOOK_SGOM_C5) # <-- ... or this should fail.

ValueError: no symbol matches

### (d) Round-Trip Tests

Source coding algorithms can contain very subtle errors that wouldn't show up in the minimal unit tests we've tested so far.
It is generally a good idea to implement more elaborate tests.
This is easy to do, now that you have both an encoder and a decoder: generate some long-ish sequence of random symbols.
Then encode and decode them and verify that the decoder reconstructs the original message.
Always remember the random number seed, so that any errors you might find are reproducible.
Otherwise, you'll have a hard time debugging them.

### (e) Improvements

You should have arrived at an implementation that works in principle but that is quite inefficient: even when ignoring for now our inefficient representation of bit strings as lists of booleans, the decoder has an expected run time of $\theta(k(\mathbf x) \times |\mathfrak X|)$ since it performs a linear search through all code words for each decoded symbol.

How could you speed up the decoding algorithm?
Think about various data structures in which you could preprocess the `codebook` so that inverting it becomes computationally cheaper.

#### Proposed Solution:

Two ideas (there are probably more):

- Create a *hash map* that inverts `codebook`, i.e., that maps from code words to symbols. Then, when iterating over all code words, do it by iterating over all code word *lengths* and look up whether `encoced[position:position+length]` is in the hash map. This reduces the complexity to $\theta(k(\mathbf x) \ell_\text{max})$ in the worst case, where $\ell_\text{max}$ is the length of the longest code word in the code book.
- Create a *trie* for the code book and then use the bits in the encoded message to walk from the root of the trie to a leaf. This reduces the complexity to $\theta(|C^*(\mathbf x)|)$.

## Problem 0.4: Refresher on Priority Heaps

This problem is very brief.
Its sole purpose is to familiarize yourself with how you can use a *priority heap* in python.
You will need a priority heap when you implement the Huffman coding algorithm in Problem 1.2 on the next problem set.

### (a) The `heapq` Python Module

An implementation of a priority heap (more precisely, a binary heap) is available in the Python `heapq` module.
Read the following demo code, verify that it runs, and make sure you understand what it does.

In [7]:
import heapq

In [8]:
# Heaps in Python are represented as lists
my_heap = []

# To interpret `my_heap` as a binary heap, we mustn't use normal list 
# operations like `my_heap.append(...)`. Instead, to insert an item, we
# have to use the function `heapq.heappush'.
heapq.heappush(my_heap, 5)
heapq.heappush(my_heap, 9)
heapq.heappush(my_heap, 2)
heapq.heappush(my_heap, 8)
print(f"my_heap after five insertions: {my_heap}")

# To extract the smallest item from the heap, we have to call `heapq.heappop`:
smallest = heapq.heappop(my_heap)
print(f"smallest item: {smallest}")
print(f"my_heap after extracting the smallest item: {my_heap}")

# Notice that extracting the smallest item also rearranged the remaining
# items in the list. This is so that when we call `heapq.heappop` again we'll
# again get the (now) smallest item:
smallest = heapq.heappop(my_heap)
print(f"smallest item: {smallest}")
print(f"my_heap after extracting the smallest item again: {my_heap}")

my_heap after five insertions: [2, 8, 5, 9]
smallest item: 2
my_heap after extracting the smallest item: [5, 8, 9]
smallest item: 5
my_heap after extracting the smallest item again: [8, 9]


### (b) Heap Sort

Implement the body of the following function `heapsort`.
The function takes an unsorted list of items and it should return a list of the same items in sorted order (from smallest to largest).
Don't use any built-in sorting algorithm.
Instead, call `heapq.heappush` repeatedly to push all items on a heap (or call `heapq.heapify` once, which is equivalent but more efficient) and then extract them into a sorted list, which you return.

Verify your implementation by running the unit test below.

In [9]:
def heapsort(unsorted_list):
    # TODO: fill in the function body
    #
    # SUGGESTED SOLUTION:
    heap = unsorted_list.copy()
    heapq.heapify(heap)
    # NOTE: the above two lines are equivalent to:
    #   heap = []
    #   for item in unsorted_list:
    #       heapq.heappush(heap, item)
    sorted_list = []
    while len(heap) != 0:
        sorted_list.append(heapq.heappop(heap))
    heapq.heapify
    return sorted_list

#### Unit test:

In [10]:
test_data1 = [
    2.73, 8.57, -0.29, -0.50, -0.18, 1.18, 2.71, -0.41, 7.71, 9.52, 
    3.48, 0.74, 7.71, 5.24, 3.85, 2.16, 5.71, 9.91, 0.39, 6.26]
test_data2 = [] # Test the corner case of an empty list.
test_data3 = [3.2] # Test the corner case of a list with only one entry.
test_data4 = [2, 5, 2] # Test the corner case of a list with equal entries.

assert heapsort(test_data1) == sorted(test_data1)
assert heapsort(test_data2) == sorted(test_data2)
assert heapsort(test_data3) == sorted(test_data3)
assert heapsort(test_data4) == sorted(test_data4)